In [ ]:
from minio.error import S3Error
import fitz  # PyMuPDF
from datetime import datetime
from io import BytesIO
import pandas as pd
import re
import psycopg2
import import_ipynb
from Utils import conectar_minio, guardar_df_en_minio, generar_ruta_fecha
import os
from dotenv import load_dotenv

Conexión exitosa a MinIO en localhost:9000
Buckets disponibles: ['yachay', 'yachay-bronze', 'yachay-landing']
Conexión exitosa a PostgreSQL PostgreSQL 16.0, compiled by Visual C++ build 1935, 64-bit
2025/05/06


In [ ]:
# Cargar variables del archivo .env
load_dotenv()

In [ ]:
minio_client = conectar_minio(
    endpoint=os.getenv("MINIO_ENDPOINT"),
    access_key=os.getenv("MINIO_ACCESS_KEY"),
    secret_key=os.getenv("MINIO_SECRET_KEY"),
    secure=False  # Usar HTTPS
)

Conexión exitosa a MinIO en localhost:9000
Buckets disponibles: ['yachay', 'yachay-bronze', 'yachay-landing']


In [3]:
def extract_pdf_content(pdf_document):
    """
    Extrae contenido específico de un archivo PDF ya abierto.

    Args:
        pdf_document (fitz.Document): Documento PDF abierto con PyMuPDF.

    Returns:
        dict: Diccionario con la información extraída.
    """
    extracted_data = {
        "Numero de Informe": None,
        "Nombre del Paciente": None,
        "Identificacion": None,
        "Edad": None,
        "Telefono": None,
        "Fecha de Toma de Muestra": None,
        "Fecha de Ingreso": None,
        "Fecha de Informe": None,
        "Entidad": None,
        "EPS": None,
        "Servicio": None,
        "Muestra Remitida": None,
        "Descripcion Macroscopica": None,
        "Descripcion Microscopica": None,
        "Diagnostico": [],
        "Comentario": None
    }

    try:
        # Leer cada página del PDF
        for page_number in range(pdf_document.page_count):
            page = pdf_document[page_number]
            text = page.get_text("text")
            cleaned_text = " ".join(text.split())
            
            # Extraer información utilizando búsquedas
            if "INFORME NÚMERO:" in cleaned_text:
                extracted_data["Numero de Informe"] = cleaned_text.split("INFORME NÚMERO:")[1].split()[0].strip()
            if "NOMBRE:" in cleaned_text:
                extracted_data["Nombre del Paciente"] = cleaned_text.split("NOMBRE:")[1].split("IDENTIFICACIÓN:")[0].strip()
            if "IDENTIFICACIÓN:" in cleaned_text:
                extracted_data["Identificacion"] = cleaned_text.split("IDENTIFICACIÓN:")[1].split("EDAD:")[0].strip()
            if "EDAD:" in cleaned_text:
                extracted_data["Edad"] = cleaned_text.split("EDAD:")[1].split()[0].strip()
            if "TELÉFONO:" in cleaned_text:
                extracted_data["Telefono"] = cleaned_text.split("TELÉFONO:")[1].split()[0].strip()
            if "FECHA TOMA DE MUESTRA:" in cleaned_text:
                extracted_data["Fecha de Toma de Muestra"] = cleaned_text.split("FECHA TOMA DE MUESTRA:")[1].split()[0].strip()
            if "FECHA DE INGRESO:" in cleaned_text:
                extracted_data["Fecha de Ingreso"] = cleaned_text.split("FECHA DE INGRESO:")[1].split("FECHA DE INFORME:")[0].strip()
            if "FECHA DE INFORME:" in cleaned_text:
                extracted_data["Fecha de Informe"] = cleaned_text.split("FECHA DE INFORME:")[1].split()[0].strip()
            if "ENTIDAD:" in cleaned_text:
                extracted_data["Entidad"] = cleaned_text.split("ENTIDAD:")[1].split("EPS")[0].strip()
            if "EPS:" in cleaned_text:
                extracted_data["EPS"] = cleaned_text.split("EPS:")[1].split("SERVICIO")[0].strip()
            if "SERVICIO:" in cleaned_text:
                extracted_data["Servicio"] = cleaned_text.split("SERVICIO:")[1].split()[0].strip()
            if "MUESTRA REMITIDA:" in cleaned_text:
                extracted_data["Muestra Remitida"] = cleaned_text.split("MUESTRA REMITIDA:")[1].split()[0].strip()
            if "DESCRIPCION MACROSCOPICA:" in cleaned_text:
                extracted_data["Descripcion Macroscopica"] = cleaned_text.split("DESCRIPCION MACROSCOPICA:")[1].split("DESCRIPCION MICROSCOPICA:")[0].strip()
            if "DESCRIPCION MICROSCOPICA:" in cleaned_text:
                extracted_data["Descripcion Microscopica"] = cleaned_text.split("DESCRIPCION MICROSCOPICA:")[1].split("DIAGNOSTICO:")[0].strip()
            if "DIAGNOSTICO:" in cleaned_text:
                diagnostic_section = cleaned_text.split("DIAGNOSTICO:")[1].strip().split("\n")
                extracted_data["Diagnostico"] = [line.strip() for line in diagnostic_section if line.strip()]
            if "COMENTARIO:" in cleaned_text:
                extracted_data["Comentario"] = cleaned_text.split("COMENTARIO:")[1].strip()

    except Exception as e:
        print(f"Error al extraer datos del PDF: {e}")

    return extracted_data

In [4]:
import pandas as pd
import fitz  # PyMuPDF

def process_all_pdfs_in_minio(minio_client, bucket_name, folder_path):
    """
    Procesa todos los archivos PDF en una ruta específica de un bucket MinIO.
    
    Parámetros:
    -----------
    minio_client : Minio
        Cliente MinIO ya conectado
    bucket_name : str
        Nombre del bucket donde están los archivos
    folder_path : str
        Ruta dentro del bucket (ej: 'informes/2023/')
    
    Retorna:
    --------
    pandas.DataFrame
        DataFrame con los datos extraídos de todos los PDFs
    """
    results = []
    
    try:
        # Asegurar que la ruta termine con '/'
        if not folder_path.endswith('/'):
            folder_path += '/'
        
        # Listar objetos en la ruta especificada
        objects = minio_client.list_objects(bucket_name, prefix=folder_path, recursive=True)
        
        for obj in objects:
            if obj.object_name.lower().endswith('.pdf'):
                print(f"Procesando archivo: {obj.object_name}")
                pdf_data = None
                pdf_document = None
                
                try:
                    # Obtener el objeto PDF
                    response = minio_client.get_object(bucket_name, obj.object_name)
                    pdf_data = BytesIO(response.read())
                    
                    # Abrir el PDF
                    pdf_document = fitz.open(stream=pdf_data, filetype="pdf")
                    
                    # Extraer contenido (asumiendo que tienes esta función)
                    content = extract_pdf_content(pdf_document)
                    
                    # Agregar los datos al resultado
                    results.append({
                        "Archivo": obj.object_name.split('/')[-1],  # Solo el nombre del archivo
                        "Ruta Completa": obj.object_name,
                        "Numero de Informe": content.get("Numero de Informe"),
                        "Nombre del Paciente": content.get("Nombre del Paciente"),
                        "Identificacion": content.get("Identificacion"),
                        "Edad": content.get("Edad"),
                        "Telefono": content.get("Telefono"),
                        "Fecha de Toma de Muestra": content.get("Fecha de Toma de Muestra"),
                        "Fecha de Ingreso": content.get("Fecha de Ingreso"),
                        "Fecha de Informe": content.get("Fecha de Informe"),
                        "Entidad": content.get("Entidad"),
                        "EPS": content.get("EPS"),
                        "Servicio": content.get("Servicio"),
                        "Muestra Remitida": content.get("Muestra Remitida"),
                        "Descripcion Macroscopica": content.get("Descripcion Macroscopica"),
                        "Descripcion Microscopica": content.get("Descripcion Microscopica"),
                        "Diagnostico": content.get("Diagnostico"),
                        "Comentario": content.get("Comentario"),
                    })
                    
                except Exception as e:
                    print(f"Error procesando {obj.object_name}: {e}")
                finally:
                    if pdf_document is not None:
                        pdf_document.close()
                    if 'response' in locals():
                        response.close()
                        response.release_conn()
    
    except S3Error as e:
        print(f"Error al acceder al bucket {bucket_name}: {e}")
    except Exception as e:
        print(f"Error inesperado: {e}")
    
    # Convertir resultados en un DataFrame
    return pd.DataFrame(results)

In [5]:
# Ejemplo de uso
df_resultados = process_all_pdfs_in_minio(
    minio_client=minio_client,  # Tu cliente MinIO ya conectado
    bucket_name="yachay-landing",
    folder_path="Laboratorios/Laboratorio Jimenes Ipiales"  # Ruta dentro del bucket
)

# Mostrar resultados
print(f"Se procesaron {len(df_resultados)} archivos PDF")
display(df_resultados.head())

Procesando archivo: Laboratorios/Laboratorio Jimenes Ipiales/01-23-27355204.pdf
Procesando archivo: Laboratorios/Laboratorio Jimenes Ipiales/02-23-1085915998.pdf
Procesando archivo: Laboratorios/Laboratorio Jimenes Ipiales/03-23-1085945182.pdf
Procesando archivo: Laboratorios/Laboratorio Jimenes Ipiales/04-23-37120149.pdf
Procesando archivo: Laboratorios/Laboratorio Jimenes Ipiales/05-23-1085918055.pdf
Se procesaron 5 archivos PDF


,Archivo,Ruta Completa,Numero de Informe,Nombre del Paciente,Identificacion,Edad,Telefono,Fecha de Toma de Muestra,Fecha de Ingreso,Fecha de Informe,Entidad,EPS,Servicio,Muestra Remitida,Descripcion Macroscopica,Descripcion Microscopica,Diagnostico,Comentario
0,01-23-27355204.pdf,Laboratorios/Laboratorio Jimenes Ipiales/01-23...,01-23,GRACIELA ESTHER LOZA ALVAREZ,CC 27355204,59,None,29/12/2022,10/12/2022,17/01/2023,None,EMSSANAR,GINECOLOGIA,UTERO,Rotulado “útero” en formol se recibe útero des...,En los cortes histológicos se reconoce cérvix ...,[UTERO; HISTERECTOMIA SIMPLE:  CERVICITIS CRO...,None
1,02-23-1085915998.pdf,Laboratorios/Laboratorio Jimenes Ipiales/02-23...,02-23,YULI VIVIANA CALPA ARCINIEGAS,CC 1085915998,33,None,30/12/2022,10/01/2023,17/01/2023,None,MALLAMAS,GINECOLOGIA,TUBAS,se recibe dos muestras: 1. Rotulado “fimbrias”...,A) Los cortes muestran pared de trompas uterin...,[1. SEGMENTO DE TROMPAS UTERINAS; POMEROY:  D...,None
2,03-23-1085945182.pdf,Laboratorios/Laboratorio Jimenes Ipiales/03-23...,003-2023,DANY PAOLA CHACUA PUERCHAMBUD,CC 1085945182,25,None,29/12/2022,10/01/2023,12/01/2023,None,MALLAMAS,GINECOLOGIA,FIMBRIAS,"Rotulado “fimbrias”, en formol se recibe dos s...",Los cortes muestran pared de trompas uterinas ...,[SEGMENTO DE TROMPAS UTERINAS; POMEROY:  DENT...,None
3,04-23-37120149.pdf,Laboratorios/Laboratorio Jimenes Ipiales/04-23...,04-2023,STELLA ISAURA YANDUN PANTOJA,CC 37120149,44,None,29/12/2022,10/01/2023,12/01/2023,None,MALLAMAS,QUIROFANO,VESICULA,"Rotulado “vesícula”, en formol se recibe dos f...",Pared de vesícula biliar engrosada. Presenta d...,[VESICULA BILIAR; COLECISTECTOMIA:  COLECISTI...,None
4,05-23-1085918055.pdf,Laboratorios/Laboratorio Jimenes Ipiales/05-23...,005-2023,SANDRA VIVIANA CHAMORRO ESCOBAR,CC 1085918055,32,None,29/12/2022,10/01/2023,12/01/2023,None,SANITAS,GINECOLOGIA,FIMBRIAS,"Rotulado “fimbrias”, en formol se recibe dos s...",Los cortes muestran pared de trompas uterinas ...,[SEGMENTO DE TROMPAS UTERINAS; POMEROY:  DENT...,None


In [6]:
ruta_fecha = generar_ruta_fecha()

In [7]:
# Guardar el DataFrame en MinIO
ruta_minio = guardar_df_en_minio(
    minio_client=minio_client,  # Cliente de la conexión anterior
    df=df_resultados,
    bucket_name="yachay-bronze",
    ruta_destino=f"laboratorios/{ruta_fecha}/laboratorio Jimenes Ipiales/laboratorio_jimenes_ipiales",  # La extensión se agregará automáticamente
    formato='csv',  # También puede ser 'csv', 'json' o 'excel'
    crear_bucket=True
)

DataFrame guardado exitosamente en: yachay-bronze/laboratorios/2025/05/06/laboratorio Jimenes Ipiales/laboratorio_jimenes_ipiales.csv
